In [18]:
from cropharvest.datasets import CropHarvest
from cropharvest.inference import Inference
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier

import requests
import tempfile

DATA_DIR = "data"

!mkdir $DATA_DIR

A subdirectory or file data already exists.


In [19]:
from sklearn.ensemble import RandomForestClassifier

from cropharvest.datasets import Task, CropHarvest
from cropharvest.countries import get_country_bbox

data_dir = "data"

Togo_Bounding_Box = get_country_bbox("Togo")
Togo_Bounding_Box = Togo_Bounding_Box[0]

In [20]:
togo_task = Task(bounding_box=Togo_Bounding_Box, normalize=True)
togo_dataset = CropHarvest(data_dir, togo_task)

RandomForestClassifier(random_state=0)

In [28]:
evaluation_datasets = CropHarvest.create_benchmark_datasets(DATA_DIR)
evaluation_datasets
togo_dataset = evaluation_datasets[2]

In [21]:
for test in X:
    if len(test) !=  216:
        print("Error")

In [22]:
def count_h5(country):
    DATA_DIR = "data/features/arrays/"
    """Count the number of h5 files for a given country."""
    train = len(list(Path(DATA_DIR).glob(f"*{country}.h5")))
    test = len(list(Path(DATA_DIR).glob(f"*{country}-eval.h5")))
    return train, test

count_h5('togo')

(1272, 306)

In [66]:
import config
import json
from cropharvest.engineer import TestInstance

sample_sizes = config.DATASET_TO_SIZES['Togo']
# sample_sizes = [20]
seed_average = False


print(sample_sizes)
# seed = 0
# togo_dataset.shuffle(seed=seed)

Skip_Existing_Results = True
   
for seed in config.SHUFFLE_SEEDS:
    togo_dataset.shuffle(seed=seed)
    for sample in sample_sizes:
        print("Running sample size: ", sample)
        json_suffix = f"{togo_dataset.id}_{sample}_{seed}.json"
        nc_suffix  = f"{togo_dataset.id}_{sample}_{seed}.nc"
        average_suffix = f"{togo_dataset.id}_{sample}*.nc"

        train_x, train_y = togo_dataset.as_array(flatten_x=True, num_samples=sample)
        print(train_x.shape, train_y.shape)
        model = RandomForestClassifier(random_state=42)
        model.fit(train_x, train_y)

        for test_id, test_instance in togo_dataset.test_data(flatten_x=True):
            results_json =  "Results/RF/Togo/" + json_suffix
            results_nc =  "Results/RF/Togo/" + nc_suffix

            if Path(results_json).exists() and Skip_Existing_Results:
                print(f"Results already saved for {results_json} - skipping")

            else:
                preds = model.predict_proba(test_instance.x)
                print(preds[0:10], preds.shape)
                preds = preds[:, 1]
                print(preds[0:10], preds.shape)
                results = test_instance.evaluate_predictions(preds)
                print(results)

                with Path(results_json).open("w") as f:
                    json.dump(results, f)

                ds = test_instance.to_xarray(preds)
                ds.to_netcdf(results_nc)

        # finally, we want to get results when all the test instances are considered
                    # together
    if seed_average:
        print("Running seed average")
        all_nc_files = list(Path('Results/RF/Togo/').glob(f"*{seed}.nc"))
        print(all_nc_files)
        combined_instance, combined_preds = TestInstance.load_from_nc(all_nc_files)

        combined_results = combined_instance.evaluate_predictions(combined_preds)

        with (Path('Results/RF/Togo_Combined/') / f"Seed_combined_{json_suffix}").open("w") as f:
            json.dump(combined_results, f)
if not seed_average:
    print("Averaging using sample size")
    for sample in sample_sizes:
        average_suffix = f"{togo_dataset.id}_{sample}"
        all_nc_files = list(Path('Results/RF/Togo/').glob(f"{average_suffix}*.nc"))
        print(all_nc_files)
        combined_instance, combined_preds = TestInstance.load_from_nc(all_nc_files)

        combined_results = combined_instance.evaluate_predictions(combined_preds)

        with (Path('Results/RF/Togo_Combined/') / f"Sample_combined_{average_suffix}.json").open("w") as f:
            json.dump(combined_results, f)







[20, 50, 126, 254, 382, 508, 636, 764, 892, 1020, 1148, None]
Averaging using sample size
[WindowsPath('Results/RF/Togo/Togo_crop_20_0.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_1.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_2.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_3.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_4.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_5.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_6.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_7.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_8.nc'), WindowsPath('Results/RF/Togo/Togo_crop_20_9.nc')]
[WindowsPath('Results/RF/Togo/Togo_crop_508_0.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_1.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_2.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_3.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_4.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_5.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_6.nc'), WindowsPath('Results/RF/Togo/Togo_crop_508_7.nc'), 